In [3]:
from fastai.vision.all import *
import cv2
import skimage as sm
import skimage.io
import tifffile
import os
import numpy as np
path = Path('/notebooks/DL_paper') # replace with your own 

# Load model for testing

# loads txt file with names of files in validation dataset
valid_fnames = (path/'models/tissueModel/validTissue.txt').read_text().split('\n')
path_im = path/'train_images'
path_lbl = path/'train_masks'

# Names of class
codes = np.loadtxt(path/'models/tissueModel/codesTissue.txt', dtype=str)
fnames = get_image_files(path_im)
lbl_names = get_image_files(path_lbl)

# function to get mask using name of image
get_msk = lambda o: path/f'dat_tissue/train_masks'/f'mask{o.stem.split("g")[1]}{o.suffix}'

# 
camvid = DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                    get_items=get_image_files,
                    splitter=FileSplitter(path/'models/tissueModel/validTissue.txt'),
                    get_y=get_msk,
                    batch_tfms=[*aug_transforms(), # augmentations
                    Normalize.from_stats(*imagenet_stats)]) 
dls = camvid.dataloaders(path/f'dat_tissue/train_images', bs=4) # bs is the batch size
dls.vocab = codes
name2id = {v:k for k,v in enumerate(codes)}
void_code = name2id['Void']

# define metric to assess model performance 
def acc_camvid(inp, targ):
  targ = targ.squeeze(1)
  mask = targ != void_code
  return (inp.argmax(dim=1)[mask]==targ[mask]).float().mean() 
opt = ranger

# Makes the unet model from a resnet101 model
learn = unet_learner(dls, resnet101, metrics=acc_camvid, self_attention=True, act_cls=Mish, opt_func=opt).to_fp16()

# Load model
learn.load(f'UNetTissue')


In [6]:

# Applies model to testing data
    
def main():
    
    folder = "dat_tissue/test_images" # folder with test data images
    
    # collects names of files in the test folder
    cwd = os.getcwd()
    filenames = os.listdir(cwd + "/" + folder)
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
    if ".ipynb_checkpoints" in filenames:
        filenames.remove(".ipynb_checkpoints")
    filenames.sort()

    # runs the U-Nettissue model to test data images
    for filename in filenames:
#         print(filename)
        out = learn.predict(f'{folder}/{filename}')[2][:4]
    
        # saves predation model output
        out = np.asarray(out*255, "uint8")
        tifffile.imwrite(f"dat_tissue/test_output/pred_{filename}", out, 
            imagej=True,)

In [7]:
main()